In [1]:
%%writefile Dockerfile

FROM python:3.9-slim-buster

RUN pip install pandas==2.2.2 scikit-learn==1.4.2 prophet==1.1.5

COPY code/preprocess.py /opt/ml/code/preprocess.py
COPY code/train.py /opt/ml/code/train.py

ENV SAGEMAKER_PROGRAM preprocess.py
ENV SAGEMAKER_PROGRAM train.py
ENV PYTHONUNBUFFERED=TRUE

ENTRYPOINT ["python3"]

Overwriting Dockerfile


In [2]:
import boto3

account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.Session().region_name
ecr_repository = 'demo_prophet'
tag = ':latest'
processing_repository_uri = '{}.dkr.ecr.{}.amazonaws.com/{}'.format(account_id, region, ecr_repository + tag)

In [3]:
# Create ECR repository and push docker image
!docker build -t $ecr_repository .

Sending build context to Docker daemon    169kB
Step 1/8 : FROM python:3.9-slim-buster
 ---> c84dbfe3b8de
Step 2/8 : RUN pip install pandas==2.2.2 scikit-learn==1.4.2 prophet==1.1.5
 ---> Using cache
 ---> a46aef910c10
Step 3/8 : COPY code/preprocess.py /opt/ml/code/preprocess.py
 ---> 20a2cefcc36c
Step 4/8 : COPY code/train.py /opt/ml/code/train.py
 ---> d12c9ec2e7a9
Step 5/8 : ENV SAGEMAKER_PROGRAM preprocess.py
 ---> Running in d87f1061bc1b
Removing intermediate container d87f1061bc1b
 ---> 1674ba879f7a
Step 6/8 : ENV SAGEMAKER_PROGRAM train.py
 ---> Running in 885a56402a5a
Removing intermediate container 885a56402a5a
 ---> f2ac2750049b
Step 7/8 : ENV PYTHONUNBUFFERED=TRUE
 ---> Running in 5e7784720e76
Removing intermediate container 5e7784720e76
 ---> 87d87052cfa6
Step 8/8 : ENTRYPOINT ["python3"]
 ---> Running in 39d061a7149e
Removing intermediate container 39d061a7149e
 ---> 690e4f8f2fbc
Successfully built 690e4f8f2fbc
Successfully tagged demo_prophet:latest


In [4]:
!aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {account_id}.dkr.ecr.{region}.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [5]:
!aws ecr create-repository --repository-name $ecr_repository


An error occurred (RepositoryAlreadyExistsException) when calling the CreateRepository operation: The repository with name 'demo_prophet' already exists in the registry with id '654654222480'


In [8]:
!docker tag {ecr_repository + tag} $processing_repository_uri

In [9]:
!docker push $processing_repository_uri

The push refers to repository [654654222480.dkr.ecr.us-east-1.amazonaws.com/demo_prophet]

34ee111c: Preparing 
a2501ff9: Preparing 
d0cba51b: Preparing 
a27560c1: Preparing 
037e08b3: Preparing 
eede8d6e: Preparing 
55769c5e: Preparing 
8a51359d: Layer already exists latest: digest: sha256:4557adba62c9b52d46fd40cdf91c3fb2db225e0c6009ceb8abe4fb29e1b60877 size: 1997
